# RNN
This notebook contains scraches of RNNs

# Load Data

In [1]:
import utils

  _________________________________
| Feed me some grass! Ik heb honger |
                                 \
                                  \
                                    ^__^
                                    (oo)\_______
                                    (__)\       )\/\
                                        ||----w |
                                        ||     ||


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

###### Old data loading method

In [4]:
# dirpath_left_nopunch = "Data/Raw Data/Preliminary/Preliminary Left No Punch/"
# dirpath_right_nopunch = "Data/Raw Data/Preliminary/Preliminary Right No Punch/"
# dirpath_jab = "Data/Raw Data/Preliminary/Preliminary Jab/"
# dirpath_cross = "Data/Raw Data/Preliminary/Preliminary Cross/"
# dirpath_lh = "Data/Raw Data/Preliminary/Preliminary Lead Hook"
# dirpath_rh = "Data/Raw Data/Preliminary/Preliminary Rear Hook/"
# dirpath_lu = "Data/Raw Data/Preliminary/Preliminary Lead Uppercut/"
# dirpath_ru = "Data/Raw Data/Preliminary/Preliminary Rear Uppercut/"

# dirpath_savefig = "Figures/"

In [5]:
# dirpath_left_nopunch = "Data/Raw Data/Preliminary/Preliminary Left No Punch/"
# dirpath_right_nopunch = "Data/Raw Data/Preliminary/Preliminary Right No Punch/"
# dirpath_jab = "Data/Raw Data/Preliminary/Preliminary Jab/"
# dirpath_cross = "Data/Raw Data/Preliminary/Preliminary Cross/"
# dirpath_lh = "Data/Raw Data/Preliminary/Preliminary Lead Hook"
# dirpath_rh = "Data/Raw Data/Preliminary/Preliminary Rear Hook/"
# dirpath_lu = "Data/Raw Data/Preliminary/Preliminary Lead Uppercut/"
# dirpath_ru = "Data/Raw Data/Preliminary/Preliminary Rear Uppercut/"

# dirpath_savefig = "Figures/"

In [6]:
# # Left hand data without punch
# pre_lft_nopunch_acc = pd.read_csv(os.path.join(dirpath_left_nopunch,"Accelerometer.csv"))
# pre_lft_nopunch_gyr = pd.read_csv(os.path.join(dirpath_left_nopunch,"Gyroscope.csv"))
# pre_lft_nopunch_bar = pd.read_csv(os.path.join(dirpath_left_nopunch,"Barometer.csv"))

# # Right hand data without punch
# pre_rght_nopunch_acc = pd.read_csv(os.path.join(dirpath_right_nopunch,"Accelerometer.csv"))
# pre_rght_nopunch_gyr = pd.read_csv(os.path.join(dirpath_right_nopunch,"Gyroscope.csv"))
# pre_rght_nopunch_bar = pd.read_csv(os.path.join(dirpath_right_nopunch,"Barometer.csv"))

# # Jab data
# pre_jab_acc = pd.read_csv(os.path.join(dirpath_jab,"Accelerometer.csv"))
# pre_jab_gyr = pd.read_csv(os.path.join(dirpath_jab,"Gyroscope.csv"))
# pre_jab_bar = pd.read_csv(os.path.join(dirpath_jab,"Barometer.csv"))

# # Cross data
# pre_cross_acc = pd.read_csv(os.path.join(dirpath_cross,"Accelerometer.csv"))
# pre_cross_gyr = pd.read_csv(os.path.join(dirpath_cross,"Gyroscope.csv"))
# pre_cross_bar = pd.read_csv(os.path.join(dirpath_cross,"Barometer.csv"))

# # Lead Hook data
# pre_lh_acc = pd.read_csv(os.path.join(dirpath_lh,"Accelerometer.csv"))
# pre_lh_gyr = pd.read_csv(os.path.join(dirpath_lh,"Gyroscope.csv"))
# pre_lh_bar = pd.read_csv(os.path.join(dirpath_lh,"Barometer.csv"))

# # Rear Hook data
# pre_rh_acc = pd.read_csv(os.path.join(dirpath_rh,"Accelerometer.csv"))
# pre_rh_gyr = pd.read_csv(os.path.join(dirpath_rh,"Gyroscope.csv"))
# pre_rh_bar = pd.read_csv(os.path.join(dirpath_rh,"Barometer.csv"))

# # Lead Uppercut data
# pre_lu_acc = pd.read_csv(os.path.join(dirpath_lu,"Accelerometer.csv"))
# pre_lu_gyr = pd.read_csv(os.path.join(dirpath_lu,"Gyroscope.csv"))
# pre_lu_bar = pd.read_csv(os.path.join(dirpath_lu,"Barometer.csv"))

# # Rear Uppercut data
# pre_ru_acc = pd.read_csv(os.path.join(dirpath_ru,"Accelerometer.csv"))
# pre_ru_gyr = pd.read_csv(os.path.join(dirpath_ru,"Gyroscope.csv"))
# pre_ru_bar = pd.read_csv(os.path.join(dirpath_ru,"Barometer.csv"))

In [7]:
# # Use dictionary to store all dataframe to simplify the preprocessing later
# dict_pre_data = {
#     "pre_lft_nopunch_acc" : pre_lft_nopunch_acc,
#     "pre_lft_nopunch_gyr" : pre_lft_nopunch_gyr,
#     "pre_lft_nopunch_bar" : pre_lft_nopunch_bar,
#     "pre_rght_nopunch_acc" : pre_rght_nopunch_acc,
#     "pre_rght_nopunch_gyr" : pre_rght_nopunch_gyr,
#     "pre_rght_nopunch_bar" : pre_rght_nopunch_bar,    
#     "pre_jab_acc" : pre_jab_acc,
#     "pre_jab_gyr" : pre_jab_gyr,
#     "pre_jab_bar" : pre_jab_bar,    
#     "pre_cross_acc" : pre_cross_acc,
#     "pre_cross_gyr" : pre_cross_gyr,
#     "pre_cross_bar" : pre_cross_bar,
#     "pre_lh_acc" : pre_lh_acc,
#     "pre_lh_gyr" : pre_lh_gyr,
#     "pre_lh_bar" : pre_lh_bar,
#     "pre_rh_acc" : pre_rh_acc,
#     "pre_rh_gyr" : pre_rh_gyr,
#     "pre_rh_bar" : pre_rh_bar,
#     "pre_lu_acc" : pre_lu_acc,
#     "pre_lu_gyr" : pre_lu_gyr,
#     "pre_lu_bar" : pre_lu_bar,
#     "pre_ru_acc" : pre_ru_acc,
#     "pre_ru_gyr" : pre_ru_gyr,
#     "pre_ru_bar" : pre_ru_bar,}

In [8]:
# # Import the transformed dataset

# ##### Change the filepaths manually according to the path on your own device if necessary.

# dict_transformed_df = dict()

# for transformed_df in os.listdir("Data/Preprocessed Data/"):
#     df_name = transformed_df.replace(".csv", "").replace("transformed_","")
#     df_content = pd.read_csv(os.path.join("Data/Preprocessed Data/",transformed_df))
#     dict_transformed_df.setdefault(df_name, df_content)

###### New Data loading method (copied from KNN.ipynb)

In [9]:
### Naming explanation:
# punchtype_subject No._run_number_datatype
# cross_01_01_Gyroscope: Gyroscope data of the 1st Subject collected in 1st run

dirpath_experimental_data = "Data/Raw Data/Experimental"
dict_exp_data = dict()


# Walk through each folder in Experimental Data Folder 
for foldername in os.listdir(dirpath_experimental_data):
    
    # punch_name = "_".join(foldername.split("_")[:2])

    # Walk through each file in each folder. Fetch and load the Linear Acceleration data and Gyroscope data
    for filename in os.listdir(os.path.join(dirpath_experimental_data,foldername)):
        
        if any([filename.startswith("Gyroscope"), filename.startswith("Linear")]):

            data_name = str(foldername) + "_" + str(filename)
            
            # Modify the file name to make the format consistent
            data_name = "_".join(data_name.replace(".csv","").replace("Accelerometer", "Acceleration").replace("sub-", "").replace("run-","").split("_"))

            # Assign each csv as a DataFrame to a dictionary
            data_content = pd.read_csv(os.path.join(dirpath_experimental_data, foldername, filename))
            dict_exp_data.setdefault(data_name, data_content)

print(dict_exp_data.keys())

dict_keys(['cross_01_01_Gyroscope', 'cross_01_01_Linear Acceleration', 'cross_02_01_Gyroscope', 'cross_02_01_Linear Acceleration', 'cross_03_01_Gyroscope', 'cross_03_01_Linear Acceleration', 'jab_01_01_Gyroscope', 'jab_01_01_Linear Acceleration', 'jab_02_01_Gyroscope', 'jab_02_01_Linear Acceleration', 'jab_03_01_Gyroscope', 'jab_03_01_Linear Acceleration', 'lefthook_01_01_Gyroscope', 'lefthook_01_01_Linear Acceleration', 'lefthook_02_01_Gyroscope', 'lefthook_02_01_Linear Acceleration', 'lefthook_03_01_Gyroscope', 'lefthook_03_01_Linear Acceleration', 'leftnopunch_01_01_Gyroscope', 'leftnopunch_01_01_Linear Acceleration', 'leftnopunch_02_01_Gyroscope', 'leftnopunch_02_01_Linear Acceleration', 'leftnopunch_03_01_Gyroscope', 'leftnopunch_03_01_Linear Acceleration', 'leftuppercut_02_01_Gyroscope', 'leftuppercut_02_01_Linear Acceleration', 'leftuppercut_03_01_Gyroscope', 'leftuppercut_03_01_Linear Acceleration', 'leftupper_01_01_Gyroscope', 'leftupper_01_01_Linear Acceleration', 'righthook_

In [10]:
# Changing colnames for consistency

acc_columns = {"Linear Acceleration x (m/s^2)" : "X (m/s^2)",
               "Linear Acceleration y (m/s^2)" : "Y (m/s^2)",
               "Linear Acceleration z (m/s^2)" : "Z (m/s^2)"
               }

gyr_columns = {"Gyroscope x (rad/s)" : "X (rad/s)",
               "Gyroscope y (rad/s)" : "Y (rad/s)",
               "Gyroscope z (rad/s)" : "Z (rad/s)"
               }

for df_name, df_content in dict_exp_data.items():
    
    if df_name.endswith("Linear Acceleration"):
        df_content.rename(columns=acc_columns, inplace=True)

    elif df_name.endswith("Gyroscope"):
        df_content.rename(columns=gyr_columns, inplace=True)

In [11]:
# Concatenate data: First we concatenate them vertically (by same metrics), then we concatinate them horizontally (by same type of punch)

punch_types = ['leftnopunch', 'rightnopunch','jab','cross', 'lefthook', 'righthook', 'leftuppercut','rightuppercut']


def concat_df(dict_original_data, concat_step, axis):

    result_dict = dict()

    i = 0
    j = concat_step
    
    while i < len(dict_original_data):
        concat_df_name = [key for key in dict_original_data.keys()][i]
        concat_df_content = [val for val in dict_original_data.values()][i:j]
        result_dict.setdefault(concat_df_name, pd.concat(concat_df_content, axis=axis))
    
        i += concat_step
        j += concat_step
    
    return result_dict

def simplify_df_names(dict_df, list_new_names):

    dict_with_new_dfname = dict()

    for old_name, df_content in dict_df.items():

        for new_name in list_new_names:

            if any([new_name in old_name, old_name.startswith(new_name)]):
                dict_with_new_dfname.setdefault(new_name, df_content)

    return dict_with_new_dfname

dict_df_concated = concat_df(dict_exp_data, 2, 1)

dict_df_concated_final = simplify_df_names(concat_df(dict_df_concated, 3, 0), punch_types)

In [12]:
# Label the data

# Forward fill the missing value

def dataframe_labeller(df,label):
    df["Punch Type"] = label

for df, label in list(zip([df_content for df_content in dict_df_concated_final.values()],sorted(punch_types))):
    dataframe_labeller(df, label)
    df.ffill()

for punch_type, df_content in dict_df_concated_final.items():
    print(punch_type, ":")
    print(df_content)

cross :
         Time (s)  X (rad/s)  Y (rad/s)  Z (rad/s)    Time (s)  X (m/s^2)  \
0        0.065232   0.240676  -0.045339   0.668833    0.065232   1.683050   
1        0.085103   0.451193   0.059653   0.542470    0.085103   1.408963   
2        0.105020   0.605252   0.159319   0.360182    0.105020   0.555544   
3        0.124894   0.701722   0.136949   0.213113    0.124894  -0.377898   
4        0.144788   0.704385   0.005792   0.125697    0.144788  -0.791464   
...           ...        ...        ...        ...         ...        ...   
14995  299.926293  -0.307508   2.953404  -2.347397  299.926293  -0.143904   
14996  299.946296  -0.500159   3.585149  -1.779337  299.946296   0.225561   
14997  299.966297  -0.560263   4.260557  -1.421962  299.966297  -0.275865   
14998  299.986298  -0.623602   4.781751  -1.339635  299.986298  -1.019574   
14999  300.006300  -0.804991   5.138650  -1.371697  300.006300  -1.558969   

       Y (m/s^2)  Z (m/s^2) Punch Type  
0      -1.668346  -1.43239

In [13]:
# View the raw info of transformed data

for punch_type, df_content in dict_df_concated_final.items():
    print(punch_type, ":")
    print(df_content.describe())

cross :
           Time (s)     X (rad/s)     Y (rad/s)     Z (rad/s)      Time (s)  \
count  52752.000000  52752.000000  52752.000000  52752.000000  52751.000000   
mean     182.370743      0.010967      0.063197     -0.026782    182.368512   
std      113.776164      1.692620      3.565020      3.169228    113.776089   
min        0.000183    -16.179197    -34.887180    -24.224096      0.000183   
25%       87.560418     -0.442109     -0.597692     -0.486286     87.558405   
50%      175.099950     -0.004498     -0.000909     -0.000720    175.098877   
75%      262.839930      0.495264      1.143176      0.674835    262.831805   
max      450.566186     16.443207     27.484556     25.414774    450.566186   

          X (m/s^2)     Y (m/s^2)     Z (m/s^2)  
count  52751.000000  52751.000000  52751.000000  
mean       0.498109     -1.145481     -0.665079  
std       11.058316     13.689176      7.272231  
min      -80.683165   -115.448820   -123.536062  
25%       -1.179261     -2.262

In [14]:
# Remove outliers

# Before that, we drop time and return a new dict of df

def remove_outliers(df):

    # Remove the label from the original df because string is not used for statistics
    df_no_outliers = df.iloc[:,:-1].copy()
    df_label = df["Punch Type"].unique()

    accelerometer_feature = "(m/s^2)"

    for feature in df_no_outliers.columns:
        
        # Calculate Q1 (25th percentile) and Q3 (75th percentile) of each feature
        Q1 = df_no_outliers[feature].quantile(0.25)
        Q3 = df_no_outliers[feature].quantile(0.75)
        
        # Calculate IQR
        IQR = Q3 - Q1
        
        # Define outlier criteria
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Remove outliers, but treat Acceleration and other features differently (see report for details)
        if accelerometer_feature in feature:
            df_no_outliers = df_no_outliers[~((df_no_outliers[feature] < lower_bound))]
            
        else:
            df_no_outliers = df_no_outliers[~((df_no_outliers[feature] < lower_bound) | (df_no_outliers[feature] > upper_bound))]

    # Assign the label back
    df_no_outliers["Punch Type"] = df_label[0]

    return df_no_outliers


In [15]:
def boxplot_feature(df, punch_type, figsize = (10,10), show_dist = False, savefig=False):

    """
    Make Boxplot of each feature within a same type of punch.
    If show_dist = True, then it will plot a violinplot instead. 
    """

    df_copy = df.copy()
    
    if show_dist:
        
        plt.figure(figsize=figsize)    
        sns.violinplot(df_copy)
        plt.title(f"Violinplot of features in {punch_type}")
    
        if savefig:
            plt.savefig(f"Figures\\violin_{punch_type}.png")

    else:
        plt.figure(figsize=figsize)    
        sns.boxplot(df_copy)
        plt.title(f"Boxplot of features in {punch_type}")
    
        if savefig:
            plt.savefig(f"Figures\\Boxplot_{punch_type}.png")
        
    plt.show()

# def make_violinplot_punchtype(feature, figsize = (10,10), savefig=False):
#     df_joined = pd.concat([df for df in dict_transformed_data.values()],axis=0)

#     plt.figure(figsize=figsize)
#     df_melted = df_joined.melt(id_vars='Punch Type', value_vars=[feature], var_name='Feature', value_name='Value')
    
#     plt.figure(figsize=figsize)
#     sns.violinplot(data=df_melted, x='Feature', y='Value', hue='Punch Type', split=True)
#     plt.title(f"Violinplot of {feature} across classes")
    
#     if savefig:
#         plt.savefig(f"violinplot_{feature}.png")
    
#     plt.show()


In [16]:
# Remove time from the data
dict_df_concated_NoTime = dict()

for punch_type, df_content in dict_df_concated_final.items():

    dict_df_concated_NoTime.setdefault(punch_type, df_content.drop(columns="Time (s)"))

dict_df_concated_NoTime

{'cross':        X (rad/s)  Y (rad/s)  Z (rad/s)  X (m/s^2)  Y (m/s^2)  Z (m/s^2)  \
 0       0.240676  -0.045339   0.668833   1.683050  -1.668346  -1.432390   
 1       0.451193   0.059653   0.542470   1.408963  -0.833281  -1.033995   
 2       0.605252   0.159319   0.360182   0.555544   0.084102  -0.627524   
 3       0.701722   0.136949   0.213113  -0.377898   1.033287  -0.123379   
 4       0.704385   0.005792   0.125697  -0.791464   1.543793   0.366435   
 ...          ...        ...        ...        ...        ...        ...   
 14995  -0.307508   2.953404  -2.347397  -0.143904   1.580076   1.579204   
 14996  -0.500159   3.585149  -1.779337   0.225561   0.673862  -0.569482   
 14997  -0.560263   4.260557  -1.421962  -0.275865   0.650340  -0.795494   
 14998  -0.623602   4.781751  -1.339635  -1.019574   1.862951  -0.137417   
 14999  -0.804991   5.138650  -1.371697  -1.558969   2.324484  -1.287985   
 
       Punch Type  
 0          cross  
 1          cross  
 2          cross

In [17]:
### Create 2 versions of aggregated data with window size of 50 and 100. Both of them are groupped by mean absolute value

print(help(utils.aggregate_data))

dict_df_transformed_50 = dict()
dict_df_transformed_100 = dict()

for punch_type, df_content in dict_df_concated_NoTime.items():
    dict_df_transformed_50.setdefault(punch_type, utils.aggregate_data(df_content, 50, punch_type))
    dict_df_transformed_100.setdefault(punch_type, utils.aggregate_data(df_content, 100, punch_type))

Help on function aggregate_data in module utils:

aggregate_data(df, window, label=None)
    Returns an aggregated dataframe on a given window size by the mean absolut value of each window.
    
        Discards the remainders that have less datapoints than the window size. Usually the 'tail' of the dataframe.
        Does not take the influence of time series into account. Usually applied for data preprocessing
            for a classical ML algorithm.
    
    :param df: The Raw, Unprocessed Data (collected by Phyphox or other device)
    :param int window: The window size of data aggregation.
    :(Optional) param str label: A label to assign to the aggregated data. None by default.
        If None, unlabelled dataframe will be returned.
    
    :returns df_agg: A dataframe with features aggregated by the mean absolut value of each window.

None


In [18]:
# Remove outliers
dict_transformed_50_clean = dict()
dict_transformed_100_clean = dict()

for df_name, df_content in dict_df_transformed_50.items():
    dict_transformed_50_clean.setdefault(df_name, remove_outliers(df_content))

for df_name, df_content in dict_df_transformed_100.items():
    dict_transformed_100_clean.setdefault(df_name, remove_outliers(df_content))

In [19]:
# Convert the clean data into a complete dataset and shuffle it
# Can be done with train_test_split as well, but I did iy myself anyway

df_clean_window50 = pd.concat([df_content for df_content in dict_transformed_50_clean.values()], axis = 0)
df_clean_window100 = pd.concat([df_content for df_content in dict_transformed_100_clean.values()], axis = 0)

df_clean_window50 = df_clean_window50.sample(frac=1).reset_index(drop=True)
df_clean_window100 = df_clean_window100.sample(frac=1).reset_index(drop=True)

df_clean_window100

,X (rad/s),Y (rad/s),Z (rad/s),X (m/s^2),Y (m/s^2),Z (m/s^2),Punch Type
0,1.532,2.576,2.410,12.291,10.417,9.294,lefthook
1,1.676,2.500,3.264,13.336,11.911,10.739,lefthook
2,0.343,1.493,1.072,3.238,3.065,2.074,jab
3,0.143,0.254,0.312,1.527,1.604,1.830,leftnopunch
4,2.382,1.440,1.654,4.110,6.894,6.368,righthook
...,...,...,...,...,...,...,...
3637,2.135,1.420,1.531,2.290,7.219,6.024,lefthook
3638,0.314,0.114,0.146,0.278,0.662,1.080,righthook
3639,2.199,1.664,1.793,3.266,8.209,6.729,lefthook
3640,0.924,2.216,1.581,5.960,7.511,3.887,jab


In [20]:
# # Save the two cleaned dataframes

# dirpath_preprocessed_data = "Data/Preprocessed Data/"

# df_clean_window50.to_csv(os.path.join(dirpath_preprocessed_data, "clean_data_window50.csv"))
# df_clean_window100.to_csv(os.path.join(dirpath_preprocessed_data, "clean_data_window100.csv"))

# LSTM

An optimized version of RNN that evaluates the importance of information passed to the Network by Tanh and Sigmoid function and selectively filters out the information that are considered less important. This method is famous for its ability in avoiding vanishing/exploring Gradient Problem.

LSTM pipeline:
- Data Preprocessing: Standardization and Convering data into sequences of 5 datapoints. LSTM will study the pattern in these 5 sequences.
- - e.g.: We give a sequence of 5 jabs to LSTM, LSTM will know that *On this is how Jabs look like!* And then learn to make this prediction
- Train-test split
- One-hot encode the label
- Train the model
- Evaluation

### Window of 50

###### Data Preprocessing: convert datapoints into sequences

###### To ensure labels are correctly assigned to each sequence, we convert the clean but unstacked dataframes in dict_transformed_50_clean
###### to sequences, then we stack and shuffle them

In [21]:
def drop_tails(df, seq_length):
    
    df_length = len(df) // seq_length * seq_length
    return df.iloc[:df_length]

# Convert data into sequences
def create_sequential_data(df, sequence_length):
    x = []
    y = []
    features = df.iloc[:,:-1]
    labels = df.iloc[:,-1]

    num_sequences = len(df) // sequence_length

    sequence_start_idx = 0 
    sequence_end_idx = sequence_length

    while num_sequences > 0:
        x.append(features[sequence_start_idx:sequence_end_idx])
        y.append(labels.iloc[sequence_start_idx])

        # print(num_sequences, sequence_start_idx, sequence_end_idx)
        num_sequences -= 1
        sequence_start_idx += sequence_length
        sequence_end_idx += sequence_length 

    return np.array(x), np.array(y)

In [22]:
seq_length = 5

seq_data_50 = []
seq_label_50 = []

for df_name, df_content in dict_transformed_50_clean.items():
    sub_seq_data, sub_seq_label = create_sequential_data(drop_tails(df_content, seq_length), seq_length)
    seq_data_50.append(sub_seq_data)
    seq_label_50.append(sub_seq_label)

seq_data_50 = np.vstack(tuple([arr for arr in seq_data_50]))
seq_label_50 = np.vstack(tuple([arr.reshape(-1,1) for arr in seq_label_50]))

In [23]:
# Encode the label
label_encoder = OneHotEncoder(sparse_output=False)

seq_label_50 = label_encoder.fit_transform(seq_label_50)

In [24]:
# Perform train test split 
# This time we will let train_test_split to do the shuffling task

x_train_50, x_test_50, y_train_50, y_test_50 = train_test_split(seq_data_50, seq_label_50, test_size=0.3, random_state=50)

# scaler = MinMaxScaler()
# x_train_50 = scaler.fit_transform(x_train_50.reshape(-1,1))
# x_test_50 = scaler.transform(x_test_50.reshape(-1,1))


In [25]:
# To ensure the quality of training and to avoid bias, carefully check if there exists data leakage
# Data leakage happens when training and test dataset have same elements.
# The follwoing function will return False if there is no data leakage.

def check_data_leakage_sequence(df_train, df_test, show_leaked_data = False):

    set_train = set()
    set_test = set()
    datatype = type(df_train)
    
    for sequence in df_train:
        set_train.add(tuple(sequence.ravel()))

    for sequence in df_test:
        set_test.add(tuple(sequence.ravel()))

    leaked_data = set_train.intersection(set_test)

    if len(leaked_data) == 0:
        print("No data leakage!")
        return False

    else:
        print("Data leaked!")

        if show_leaked_data:
            print(leaked_data)

        return True

In [26]:
check_data_leakage_sequence(x_train_50, x_test_50)

No data leakage!


False

### Construct the model

In [27]:
lstm_model = Sequential()

lstm_model.add(LSTM(units=100, input_shape=(x_train_50.shape[1], x_train_50.shape[2])))

lstm_model.add(Dropout(0.5))

lstm_model.add(Dense(y_train_50.shape[1], activation='softmax'))

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               42800     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 8)                 808       
                                                                 
Total params: 43608 (170.34 KB)
Trainable params: 43608 (170.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
history = lstm_model.fit(x_train_50, y_train_50, epochs=1000, batch_size=5, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/1000


168/168 [==============================] - 2s 4ms/step - loss: 1.7042 - accuracy: 0.3349 - val_loss: 1.1383 - val_accuracy: 0.5694
Epoch 2/1000
168/168 [==============================] - 0s 3ms/step - loss: 1.0974 - accuracy: 0.5347 - val_loss: 0.8669 - val_accuracy: 0.6459
Epoch 3/1000
168/168 [==============================] - 0s 2ms/step - loss: 0.9359 - accuracy: 0.6065 - val_loss: 0.7832 - val_accuracy: 0.6651
Epoch 4/1000
168/168 [==============================] - 0s 2ms/step - loss: 0.8612 - accuracy: 0.6112 - val_loss: 0.7282 - val_accuracy: 0.6651
Epoch 5/1000
168/168 [==============================] - 0s 2ms/step - loss: 0.8234 - accuracy: 0.6280 - val_loss: 0.6776 - val_accuracy: 0.6986
Epoch 6/1000
168/168 [==============================] - 0s 2ms/step - loss: 0.7820 - accuracy: 0.6519 - val_loss: 0.6985 - val_accuracy: 0.6986
Epoch 7/1000
168/168 [==============================] - 0s 2ms/step - loss: 0.7229 - accuracy: 0.6699 - val_loss: 0.6623 - val_accurac

In [30]:
# Assuming x_test and y_test are your unseen test data
test_loss, test_accuracy = lstm_model.evaluate(x_test_50, y_test_50)
print(f"Test Accuracy: {test_accuracy:.4f}")

15/15 [==============================] - 0s 2ms/step - loss: 0.3576 - accuracy: 0.8552
Test Accuracy: 0.8552


In [31]:
from sklearn.metrics import confusion_matrix, classification_report

# Predictions on test data
y_pred = lstm_model.predict(x_test_50)
# Convert predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
# Convert one-hot encoded true labels to class labels
y_true_classes = np.argmax(y_test_50, axis=1)

# Confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:")
print(cm)

# Classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes))


15/15 [==============================] - 0s 1ms/step
Confusion Matrix:
[[47 12  0  0  0  0  0  0]
 [ 3 62  0  0  0  0  0  0]
 [ 0  0 35  0 10  8  0  1]
 [ 0  0  0 48  0  0  5  0]
 [ 1  0  1  0 48  6  0  0]
 [ 0  0  1  0  0 38  0  9]
 [ 0  0  0  6  0  0 44  0]
 [ 0  0  0  0  1  1  0 62]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.80      0.85        59
           1       0.84      0.95      0.89        65
           2       0.95      0.65      0.77        54
           3       0.89      0.91      0.90        53
           4       0.81      0.86      0.83        56
           5       0.72      0.79      0.75        48
           6       0.90      0.88      0.89        50
           7       0.86      0.97      0.91        64

    accuracy                           0.86       449
   macro avg       0.86      0.85      0.85       449
weighted avg       0.86      0.86      0.85       449



### Window of 100

In [32]:
# Convert data into sequences

seq_length = 5

seq_data_100 = []
seq_label_100 = []

for df_name, df_content in dict_transformed_100_clean.items():
    sub_seq_data, sub_seq_label = create_sequential_data(drop_tails(df_content, seq_length), seq_length)
    seq_data_100.append(sub_seq_data)
    seq_label_100.append(sub_seq_label)

seq_data_100 = np.vstack(tuple([arr for arr in seq_data_100]))
seq_label_100 = np.vstack(tuple([arr.reshape(-1,1) for arr in seq_label_100]))

In [33]:
# Encode the label
label_encoder = OneHotEncoder(sparse_output=False)

seq_label_100 = label_encoder.fit_transform(seq_label_100)

In [34]:
# Perform train test split 
# This time we will let train_test_split to do the shuffling task

x_train_100, x_test_100, y_train_100, y_test_100 = train_test_split(seq_data_100, seq_label_100, test_size=0.3, random_state=50)

# scaler = MinMaxScaler()
# x_train_100 = scaler.fit_transform(x_train_100.reshape(-1,1))
# x_test_100 = scaler.transform(x_test_100.reshape(-1,1))


In [35]:
check_data_leakage_sequence(x_train_100, x_test_100)

No data leakage!


False

In [36]:
lstm_model = Sequential()

lstm_model.add(LSTM(units=100, input_shape=(x_train_100.shape[1], x_train_100.shape[2])))

lstm_model.add(Dropout(0.5))

lstm_model.add(Dense(y_train_100.shape[1], activation='softmax'))

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               42800     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 8)                 808       
                                                                 
Total params: 43608 (170.34 KB)
Trainable params: 43608 (170.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
history = lstm_model.fit(x_train_100, y_train_100, epochs=1000, batch_size=5, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/1000
82/82 [==============================] - 2s 6ms/step - loss: 1.9104 - accuracy: 0.2118 - val_loss: 1.6360 - val_accuracy: 0.4608
Epoch 2/1000
82/82 [==============================] - 0s 3ms/step - loss: 1.4828 - accuracy: 0.3892 - val_loss: 1.2111 - val_accuracy: 0.4804
Epoch 3/1000
82/82 [==============================] - 0s 3ms/step - loss: 1.1485 - accuracy: 0.5320 - val_loss: 0.9735 - val_accuracy: 0.5784
Epoch 4/1000
82/82 [==============================] - 0s 3ms/step - loss: 0.9649 - accuracy: 0.5665 - val_loss: 0.8530 - val_accuracy: 0.6667
Epoch 5/1000
82/82 [==============================] - 0s 3ms/step - loss: 0.8603 - accuracy: 0.6133 - val_loss: 0.9430 - val_accuracy: 0.5392
Epoch 6/1000
82/82 [==============================] - 0s 3ms/step - loss: 0.8036 - accuracy: 0.6650 - val_loss: 0.6776 - val_accuracy: 0.8137
Epoch 7/1000
82/82 [==============================] - 0s 3ms/step - loss: 0.8234 - accuracy: 0.6453 - val_loss: 0.7606 - val_accuracy: 0.7353
Epoch 

In [39]:
# Assuming x_test and y_test are your unseen test data
test_loss, test_accuracy = lstm_model.evaluate(x_test_100, y_test_100)
print(f"Test Accuracy: {test_accuracy:.4f}")

7/7 [==============================] - 0s 2ms/step - loss: 0.5057 - accuracy: 0.7615
Test Accuracy: 0.7615


In [40]:
# Assuming x_test and y_test are your unseen test data
test_loss, test_accuracy = lstm_model.evaluate(x_test_100, y_test_100)
print(f"Test Accuracy: {test_accuracy:.4f}")

7/7 [==============================] - 0s 2ms/step - loss: 0.5057 - accuracy: 0.7615
Test Accuracy: 0.7615


In [41]:
from sklearn.metrics import confusion_matrix, classification_report

# Predictions on test data
y_pred = lstm_model.predict(x_test_100)
# Convert predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
# Convert one-hot encoded true labels to class labels
y_true_classes = np.argmax(y_test_100, axis=1)

# Confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:")
print(cm)

# Classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes))


7/7 [==============================] - 0s 1ms/step
Confusion Matrix:
[[21  5  0  0  0  0  0  0]
 [ 0 27  0  0  0  0  0  0]
 [ 0  0 20  0  7  0  0  1]
 [ 0  0  0 20  0  0  3  0]
 [ 0  0  6  0 23  0  0  3]
 [ 0  0  5  0  4 18  0  3]
 [ 0  0  1  8  0  0 13  0]
 [ 0  1  0  0  1  4  0 24]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.81      0.89        26
           1       0.82      1.00      0.90        27
           2       0.62      0.71      0.67        28
           3       0.71      0.87      0.78        23
           4       0.66      0.72      0.69        32
           5       0.82      0.60      0.69        30
           6       0.81      0.59      0.68        22
           7       0.77      0.80      0.79        30

    accuracy                           0.76       218
   macro avg       0.78      0.76      0.76       218
weighted avg       0.77      0.76      0.76       218

